In [1]:
# logical change of basis
# example: in dual-rail 1Q gates are created by 2Q interactions

# |0>L = |01>, |1>L = |10>
# |e1> = |00>, |e2> = |11>

# Hadamard
# as a density matrix, includes implicit error states stay in error states
# H = 1/2 [|0><0|L + |0><1|L + |1><0|L - |1><1|L] (+ |e1><e1| + |e2><e2|)

# becomes
# = 1/2 [|01><01| + |01><10| + |10><01| - |10><10|] + |00><00| + |11><11|

# as an operator
# = [[1, 0, 0, 0], [0,1/sqrt(2),1/sqrt(2),0], [0,1/sqrt(2), -1/sqrt(2),0], [0,0,0,1]]

# convert to weylchamber coordinate
# -> (1/2, 1/4, 1/4)

# repeat....
# for some encoding, define some map between nQ -> (n+1)Q gates.

In [5]:
# TODO define the logical encoding using an Operator
from qutip import *
import numpy as np

# |0>
zero = basis(2, 0)
# |1>
one = basis(2, 1)
# |+_{gf}>
p_gf = (basis(3, 0) + basis(3, 2)).unit()
# |-_{gf}>
m_gf = (basis(3, 0) - basis(3, 2)).unit()

# dual-rail encoding:
#   U = |01><0| + |10><1|
U = tensor(zero, one) * zero.dag() + tensor(one, zero) * one.dag()

# NOTE what is this kind of operator called? Isometry?

## more encodings:

# some transmon qudit encoding:
# U = |g+f, g+f><0| + |g-f, g-f><1|
logical0 = tensor(p_gf, p_gf)
logical1 = tensor(m_gf, m_gf)
U = logical0 * zero.dag() + logical1 * one.dag()

# including error states
# phase flips errors E = |-+><-+| + |+-><+-|
# ancilla qubits (x, y) for distinguishing between the two types of errors

# NOTE is this form of an Isometry valid?
# idea is that it is simpler to leave out the error states until they are actually needed
# basically, the no error flag qubit is implicit
# U = |g+f, g+f><0| + |g-f, g-f><1| + |++x><+-| + |--y><-+|

# NOTE how to write expression with error (channel) E?


# SNAIL concatended erasure encoding:
# U = |g+f, g+f, g+f><0| + |g-f, g-f, g-f><1|
logical0 = tensor(p_gf, p_gf, p_gf)
logical1 = tensor(m_gf, m_gf, m_gf)
U = logical0 * zero.dag() + logical1 * one.dag()

# including error states
# two ancillas
## getting confused, need to review my previous notes to do this setup correctly


class LogicalBasis:
    """Logical basis for a qubit.

    The logical basis is defined as:
        |0>_L = zero_ket
        |1>_L = one_ket

    The two states are required to be orthogonal.
    """

    def __init__(self, zero_ket, one_ket):
        self.zero_ket = zero_ket
        self.one_ket = one_ket

        # require that the two states are orthogonal
        assert np.isclose(zero_ket.overlap(one_ket), 0)

        @property
        def basis_change_op(self):
            return self.zero_ket * zero.dag() + self.one_ket * one.dag()


class DualRail(LogicalBasis):
    """Dual-rail encoding.

    The logical basis is defined as:
        |0>_L = |01>
        |1>_L = |10>
    """

    def __init__(self):
        logical0 = tensor(basis(2, 0), basis(2, 1))
        logical1 = tensor(basis(2, 1), basis(2, 0))
        super().__init__(logical0, logical1)


class TransmonQudit(LogicalBasis):
    """Transmon qudit encoding.

    The logical basis is defined as:
        |0>_L = |g+f, g+f, g+f>
        |1>_L = |g-f, g-f, g-f>
    """

    def __init__(self):
        logical0 = tensor(p_gf, p_gf, p_gf)
        logical1 = tensor(m_gf, m_gf, m_gf)
        super().__init__(logical0, logical1)


class SNAILConcat(LogicalBasis):
    """SNAIL concatenated erasure encoding.

    The logical basis is defined as:
        |0>_L = |g+f, g+f, g+f, g+f>
        |1>_L = |g-f, g-f, g-f, g-f>
    """

    def __init__(self):
        logical0 = tensor(p_gf, p_gf, p_gf, p_gf)
        logical1 = tensor(m_gf, m_gf, m_gf, m_gf)
        super().__init__(logical0, logical1)

In [ ]:
from qutip import Qobj, sigmax


class Ancilla:
    """Represents an ancilla qubit for error detection and correction.

    Attributes:
        detection_operator (Qobj): Operator that represents the error detection.
        correction_operator (Qobj): Operator that represents the error correction.
    """

    def __init__(self, detection_operator: Qobj, correction_operator: Qobj):
        self.detection_operator = detection_operator
        self.correction_operator = correction_operator

In [1]:
class LogicalBasis:
    """Logical basis for a qubit.

    The logical basis is defined as:
        |0>_L = zero_ket
        |1>_L = one_ket

    The two states are required to be orthogonal.
    """

    def __init__(self, zero_ket: Qobj, one_ket: Qobj, *ancillas: Ancilla):
        self.zero_ket = zero_ket
        self.one_ket = one_ket
        self.ancillas = ancillas

        # require that the two states are orthogonal
        assert np.isclose(zero_ket.overlap(one_ket), 0)

    @property
    def basis_change_op(self):
        return self.zero_ket * zero.dag() + self.one_ket * one.dag()

    def detect_error(self, error_operator: Qobj) -> Qobj:
        """Apply an error operator and return the intermediate state."""
        state = self.zero_ket  # Initial state (could be more general)
        state = error_operator * state  # Apply the error

        # Apply error detection operators
        for ancilla in self.error_ancillas:
            state = ancilla * state

        return state

    def measure_ancilla(self, state: Qobj) -> tuple:
        """Measure the ancilla qubits and return the measurement results."""
        measurement_results = []
        post_measurement_state = state.copy()

        # Simulate measurement of each ancilla
        for ancilla in self.error_ancillas:
            result = (
                ancilla.dag() * post_measurement_state
            ).norm()  # Example measurement
            measurement_results.append(result)

            # Project state based on measurement result (example projection)
            if result:
                post_measurement_state = ancilla * post_measurement_state

        return measurement_results, post_measurement_state

    def fix_error(self, state: Qobj, measurement_results: tuple) -> Qobj:
        """Apply error correction based on the measurement results."""
        corrected_state = state.copy()

        # Apply correction gates based on measurement results
        for i, result in enumerate(measurement_results):
            if result:
                # Example correction: apply X gate if bit flip detected
                correction_gate = sigmax()  # QuTiP X gate
                corrected_state = correction_gate * corrected_state

                # Additional corrections can be applied based on the encoding and error model

        return corrected_state

NameError: name 'Qobj' is not defined

In [ ]:
class TransmonQuditWithAncilla(LogicalBasis):
    """Transmon qudit encoding with a single error ancilla.

    The logical basis is defined as:
        |0>_L = |g+f, g+f, g+f>
        |1>_L = |g-f, g-f, g-f>

    Includes a single ancilla for detecting errors but not distinguishing them.
    This encoding can detect when an error has occurred but does not have enough
    information to identify the specific error.

    Error Channels:
        - Phase flips
        - Bit flips

    These errors on the hardware qubits propagate into errors in the logical qubits.
    The errors that happen with the highest probability are mapped to error ancilla
    states in the logical encoding. This type of trick, known as an erasure error,
    signals leaving the codeword Hilbert space and is inspired by concepts from
    photonic quantum computing.

    Attributes:
        ancilla (Ancilla): Ancilla for error detection without distinguishing.
    """

    def __init__(self):
        logical0 = tensor(p_gf, p_gf, p_gf)
        logical1 = tensor(m_gf, m_gf, m_gf)

        # Define a single ancilla for error detection without distinguishing
        # The detection and correction operators must be defined based on the specific error model
        ancilla = Ancilla(
            detection_operator=..., correction_operator=...
        )  # Define appropriately

        super().__init__(logical0, logical1, ancilla)

In [ ]:
class TransmonQuditWithAncilla(LogicalBasis):
    """Transmon qudit encoding with error ancillas.

    The logical basis is defined as:
        |0>_L = |g+f, g+f, g+f>
        |1>_L = |g-f, g-f, g-f>

    Includes ancillas for detecting specific errors.
    """

    def __init__(self):
        logical0 = tensor(p_gf, p_gf, p_gf)
        logical1 = tensor(m_gf, m_gf, m_gf)

        # Define ancillas for error detection and correction
        ancilla1 = Ancilla(
            detection_operator=..., correction_operator=...
        )  # Define appropriately
        ancilla2 = Ancilla(
            detection_operator=..., correction_operator=...
        )  # Define appropriately

        super().__init__(logical0, logical1, ancilla1, ancilla2)


class SNAILConcatWithAncilla(LogicalBasis):
    """SNAIL concatenated erasure encoding with error ancillas.

    The logical basis is defined as:
        |0>_L = |g+f, g+f, g+f, g+f>
        |1>_L = |g-f, g-f, g-f, g-f>

    Includes ancillas for detecting specific errors.
    """

    def __init__(self):
        logical0 = tensor(p_gf, p_gf, p_gf, p_gf)
        logical1 = tensor(m_gf, m_gf, m_gf, m_gf)

        # Define ancillas for error detection and correction
        ancilla1 = Ancilla(
            detection_operator=..., correction_operator=...
        )  # Define appropriately
        ancilla2 = Ancilla(
            detection_operator=..., correction_operator=...
        )  # Define appropriately
        ancilla3 = Ancilla(
            detection_operator=..., correction_operator=...
        )  # Define appropriately

        super().__init__(logical0, logical1, ancilla1, ancilla2, ancilla3)

In [ ]:
# start by defining the codespace using qutip
# |g+f, g+f>; |g-f, g-f> are the basis states
from qutip import *
import numpy as np

t_levels = 3

p_gf = (basis(t_levels, 0) + basis(t_levels, 2)).unit()
m_gf = (basis(t_levels, 0) - basis(t_levels, 2)).unit()

# tensor 2 qubits together to get logical qubit
logical0 = tensor(p_gf, p_gf)
logical1 = tensor(m_gf, m_gf)

# verify that the basis states are orthogonal
assert not logical0.overlap(logical1)

In [ ]:
logical1

Quantum object: dims = [[3, 3], [1, 1]], shape = (9, 1), type = ket
Qobj data =
[[ 0.5]
 [ 0. ]
 [-0.5]
 [ 0. ]
 [ 0. ]
 [ 0. ]
 [-0.5]
 [ 0. ]
 [ 0.5]]

In [ ]:
from qiskit.quantum_info import DensityMatrix

In [ ]:
from qiskit.circuit.library import HGate

DensityMatrix(HGate())

DensityMatrix([[0.5+0.j, 0.5+0.j],
               [0.5+0.j, 0.5+0.j]],
              dims=(2,))
